<p><img src="imagenes/cabecera.png" width="900" align="center"></p>

# Trabajo práctico 4: Análisis de tópicos

## Curso Procesamiento de Lenguaje Natural 

### Maestría en Tecnologías de la información



**Trabajo práctico porpuesto por:** Julio Waissman Vilanova (julio.waissman@unison.mx)

**Desarrollado por:** _Poner tu nombre y correo electrónico aquí_


En esta libreta vamos a experimentar con el análisis de tópicos a partir de un corpus en español muy interesante para realizar análisis de tópicos: Una colección de sonetos del siglo de oro español. El córpus incluye mucha información adicional (metadatos) como título, autor, y sobre todo la notación métrica del soneto. Sin embargo para este trabajo no vamos a considerar ninguna de esta información adicional.

El corpus se encuentra [en este proyecto de github](https://github.com/bncolorado/CorpusSonetosSigloDeOro) y para su uso académico lo debemos citar correctamente en la referencia:

> Navarro-Colorado, Borja; Ribes Lafoz, María, and Sánchez, Noelia (2015) "Metrical annotation of a large corpus of Spanish sonnets: representation, scansion and evaluation" 10th edition of the Language Resources and Evaluation Conference 2016 Portorož, Slovenia.[PDF](http://www.dlsi.ua.es/%7Eborja/navarro2016_MetricalPatternsBank.pdf)

Lo que vamos a hacer es un análisis similar al que se realizo en [este artículo recientemente publicado](https://www.frontiersin.org/articles/10.3389/fdigh.2018.00015/full). Toma un tiempo leyendo el artículo para ver que tipo de trabajos que se publican en el área. Igualmente, este trabajo sirve para poner en prespectiva todo lo que faltaría hacer a un trabajo como el que hacemos para que sea *publicable*.

## 1. Obtención y tratamiento del *corpus*

Para obtener el *corpus* es necesario clonar el proyecto original. Desde el punto de mentaje (esto es, la carpeta `curso-pln` centro del contenedor (aunque tambien puede ser por fuera), se usa el comando:

```
git clone https://github.com/bncolorado/CorpusSonetosSigloDeOro.git
```

Una vez clonado, es necesario extraer la información de los archivos `xml`. La generación del *corpus* sin normalizar ya la dejo desarollada.

In [ ]:
import xml.etree.ElementTree as et # Para manejar buffers con estructura xml
import os

archivos_path = "/root/curso-pln/CorpusSonetosSigloDeOro/"

def lee_soneto(archivo):
    soneto = ""
    arbol = et.parse(archivo)
    raiz = arbol.getroot()
    for poema in raiz.find('{http://www.tei-c.org/ns/1.0}text'):
        if poema.tag == '{http://www.tei-c.org/ns/1.0}body':
            for parrafo in poema:
                if parrafo.tag == '{http://www.tei-c.org/ns/1.0}lg':
                    for linea in parrafo:
                        soneto += (linea.text + '\n')
                    soneto += '\n'
    return soneto

corpus_sonetos = []
for (dirpath, dirnames, filenames) in os.walk(archivos_path):
    if not dirnames:
        for filename in filenames:
            if filename[-4:] == '.xml':
                corpus_sonetos.append(lee_soneto(dirpath + '/' + filename))
            
len(corpus_sonetos)

Ejecuta varias veces la celda de abajo para ver en forma aleatoria si los sonetos se descargaron correcatmente

In [ ]:
import random
print(corpus_sonetos[random.randint(0, 5077)])

Y por último los guardamos en un archivo `pickle` para no tener que recalcular cada vez

In [ ]:
import pickle
with open("datos/sonetos.pkl", "wb") as fp:
    pickle.dump(corpus_sonetos, fp)

## 2. Normalización de texto

Ahora es necesario normalizar el texto con fin de utilizarlo en modelado de tópicos.

El texto lo vamos a tratar con al menos los siguientes requisitos:

1. Utilizar todas las palabras en minúsculas
1. Eliminar los signos de puntuación
2. Eliminar palabras vacias
3. Eliminar las palabras que no aportan significados en la poesía. Dejar las palabras que sean:
    1. pronombres personales,
    2. sustantivos,
    3. adjetivos, y/o
    4. vebos.
    
Realiza esta normalización utilizando el módulo *spacy* con el modelo `es_core_news_sm`, pero sientete con la libertad de cargar el modelo 
`es_core_news_md` que propone *spacy*. Las instucciones para instalarlo se encuentran [aquí](https://spacy.io/models/es#section-es_core_news_md). Guarda tu corpus normalizado en la variable (lista de listas) `corpus_tratado`

In [ ]:
# INSERTA AQUÏ TU CÓDIGO
# USA CUANTAS CELDAS CONSIDERES NECESARIO

# GUARDA EL CORPUS TRATADO EN corpus_tratado

Y utiliza esta celda para probar si se normalizaron vien los sontetos

In [ ]:
i = random.randint(0, 5077)
print(i)
print(corpus_sonetos[i])
print("********************")
print(corpus_tratado[i])


## 3. Modelado de tópicos con LDA

Ahora, ya con el corpus tratado, desarrolla tu modelo LDA en *gensim*, siguiendo los siguientes pasos:

1. Selecciona el número de tópicos a modelar (por default 20)
2. Crea un diccionario con el corpus tratado, donde elimines a todas las
   palabras que no aparezcan en al menos `min_df` documentos, y las palabras que 
   aparezcan el más del `max_df` porciento de documentos (por default `min_df = 5`
   y `max_df = 0.3`).
3. Genera un corpus listo para su uso, aplicando el metodo de bolsa de palabras.
4. Aplica el método para modelar con LDA. Establece el número de iteraciones para
   el método de estimación así como el numero de pasos que realiza el algoritmo
   de optimización iterativa (por default 100 y 5, respectivamente).
   
Guarda el modelo en la variable `modelo_lda`


In [ ]:
# INSERTA AQUÍ TU CÓDIGO
# USA CUANTAS CELDAS CONSIDERES NECESARIO

# GUARDA EL MODELO EN modelo_lda

Y ahora veamos los modelos como se definen por sus primeras 5 palabras clave 

In [ ]:
for (i, topico) in modelo_lda.print_topics(num_topics=n_topicos, num_words=5):
    print(10 * "-" + "topico {}".format(i) + 20 * "-")
    print(topico)

Mejor aún, utiliza *pyLDAvis* para visualizar y analizar los tópicos desarrollados.

In [ ]:
# INSERTA AQUÏ TU CÓDIGO


## 4 Responde a las preguntas siguientes

Cada pregunta implica experimentación y reflexión, por favor extiende lo que consideres necesario tus explicaciones:

### 1. ¿Cual es el número de tópicos que consideras te ofrece una mejor separación entre ellos, y mejor significado (dejando al resto de los parámetros en valores por default)? ¿Porqué? ¿Algunos tópicos son redundantes entre si? ¿Podrías asignarle un nombre a cada tópico?

_ingresa aquí tu respuesta_

### 2. ¿Que pasa si eliminas los pronombres personales en la normalización del texto? ¿Y los verbos?¿Que pasa si lematizas? ¿Hay alguna modificación a la normalización de texto que implique una mejor distribución de tópicos (con el resto de los valores por default y el número de tópicos que seleccionaste en la pregunta 1)?

_ingresa aquí tu respuesta_

### 3. ¿Que pasa si reduces el número de iteraciones en la inferencia a 50? ¿Que pasa si la aumentas a 200? ¿Y si reduces el numero de iteraciones del algoritmo de optimización a solo 1? ¿Si lo aumentas a 100 (ten cuidado, puede tardar mucho)? ¿Consideras que estos parámetros tienen mucha influencia en el resultado del método de LDA?

_ingresa aquí tu respuesta_

### 4. ¿Que pasa si aceptas todas las palabras que aparezcan en menos del 80% de los documentos? ¿Que pasaría si limitas el vocabulario a las palabras que solamente aparezcan en el $\frac{1.5}{\text(numero\ de\ tópicos)}$ por ciento de los documentos? ¿Cual es el equilibrio que consideras que es el más acertado? 

_ingresa aquí tu respuesta_

### 5. Combina todo y obten un modelado de tópicos que consideres es interesante. Escribe tus conclusiones y los tópicos que pudiste extraer del corpus bajo análisis, así como el título que pondrías a estos (o algunos de estos).

Si tienes más de un modelo que te parece interesante, puedes utilizar los métodos de medición de coherencia de los tópicos para tomar una desición.

_ingresa aquí tu respuesta_